# 02 - Build Features for KNN Recommender
## Creating the User-Item Matrix

**Goal:** Transform our ratings data into a matrix format that KNN can use.

**What we're doing:**
1. Load the cleaned data from notebook 01
2. Create a user-item matrix (rows = movies, columns = users)
3. Create lookup tables for movie IDs and titles
4. Save everything to SQLite database

In [1]:
import pandas as pd
import numpy as np
import sqlite3

print("Libraries loaded!")

Libraries loaded!


## 1. Load Data from SQLite

In [ ]:
# connect to the database we created in notebook 01
conn = sqlite3.connect("../data/processed/movielens.db")

# load ratings and movies
ratings_df = pd.read_sql("SELECT * FROM ratings", conn)
movies_df = pd.read_sql("SELECT movie_id, title FROM movies", conn)

conn.close()

print(f"Ratings: {len(ratings_df):,} rows")
print(f"Movies: {len(movies_df):,} movies")
print(f"Users: {ratings_df['user_id'].nunique():,} users")

Ratings: 99,680 rows
Movies: 1,661 movies
Users: 943 users


In [ ]:
# quick look at the data
print("Ratings sample:")
display(ratings_df.head())

print("\nMovies sample:")
display(movies_df.head())

Ratings sample:


,user_id,movie_id,rating
0,1,1,5.0
1,1,2,3.0
2,1,3,4.0
3,1,4,3.0
4,1,5,3.0



Movies sample:


,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


## 2. Create User-Item Matrix

**Why a matrix?**
- KNN needs to compare items (movies) to each other
- Each movie becomes a vector of user ratings
- We compare these vectors to find similar movies

**Matrix shape:** (movies × users)
- Each row = one movie
- Each column = one user  
- Each cell = that user's rating for that movie (0 if not rated)

In [4]:
# Pivot table: rows=movies, columns=users, values=ratings
user_item_df = ratings_df.pivot_table(
    index='movie_id',      # rows are movies
    columns='user_id',     # columns are users
    values='rating',       # cell values are ratings
    fill_value=0           # unrated movies get 0
)

print(f"Matrix shape: {user_item_df.shape}")
print(f"  - {user_item_df.shape[0]} movies (rows)")
print(f"  - {user_item_df.shape[1]} users (columns)")

Matrix shape: (1661, 943)
  - 1661 movies (rows)
  - 943 users (columns)


In [ ]:
# look at a corner of the matrix
print("First 5 movies × first 10 users:")
user_item_df.iloc[:5, :10]

First 5 movies × first 10 users:


user_id,1,2,3,4,5,6,7,8,9,10
movie_id,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3. Check Sparsity

**Why does sparsity matter?**
- Most users haven't rated most movies (lots of zeros)
- High sparsity can affect recommendation quality
- Understanding sparsity helps explain model limitations

In [6]:
# How sparse is our matrix?
total_cells = user_item_df.shape[0] * user_item_df.shape[1]
non_zero = (user_item_df != 0).sum().sum()
sparsity = 1 - (non_zero / total_cells)

print(f"Total cells: {total_cells:,}")
print(f"Non-zero cells: {non_zero:,}")
print(f"Sparsity: {sparsity:.1%} (mostly zeros!)")

Total cells: 1,566,323
Non-zero cells: 99,680
Sparsity: 93.6% (mostly zeros!)


## 4. Create Lookup Tables

We need ways to convert between:
- Movie ID ↔ Matrix row index
- Movie title ↔ Movie ID

**Why do we need these lookups?**

- **Movie ID ↔ Matrix row index**: When KNN finds similar movies, it returns row numbers (0, 1, 2...), not movie IDs. We need to translate "row 47" back to the actual movie ID. These don't always match because some movies get removed during cleaning.

- **Movie title ↔ Movie ID**: Users type movie names like "Toy Story (1995)", but the matrix only knows movie IDs. We need to translate between them.

In [7]:
# get the movie IDs in matrix row order
movie_ids = user_item_df.index.tolist()

# create lookup table: movie_id -> row index
movie_idx_df = pd.DataFrame({
    'movie_id': movie_ids,
    'matrix_idx': range(len(movie_ids))
})

print(f"Created index mapping for {len(movie_ids)} movies")
movie_idx_df.head()

Created index mapping for 1661 movies


,movie_id,matrix_idx
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4


In [8]:
# create title lookup (lowercase for easier matching)
title_lookup_df = movies_df.copy()
title_lookup_df['title_lower'] = title_lookup_df['title'].str.lower()

print("Title lookup sample:")
title_lookup_df.head()

Title lookup sample:


,movie_id,title,title_lower
0,1,Toy Story (1995),toy story (1995)
1,2,GoldenEye (1995),goldeneye (1995)
2,3,Four Rooms (1995),four rooms (1995)
3,4,Get Shorty (1995),get shorty (1995)
4,5,Copycat (1995),copycat (1995)


## 5. Save Everything to SQLite

We'll save:
1. **matrix_df** - The full user-item matrix as a table
2. **movie_idx_df** - Maps movie_id to matrix row index
3. **title_lookup_df** - Maps titles to movie_id

In [9]:
# reset index so movie_id becomes a column
matrix_df = user_item_df.reset_index()

# rename columns to be SQL-friendly (user_1, user_2, etc.)
new_cols = ['movie_id'] + [f'user_{c}' for c in matrix_df.columns[1:]]
matrix_df.columns = new_cols

print(f"Matrix table shape: {matrix_df.shape}")
matrix_df.head()

Matrix table shape: (1661, 944)


,movie_id,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,...,user_934,user_935,user_936,user_937,user_938,user_939,user_940,user_941,user_942,user_943
0,1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# save to SQLite
conn = sqlite3.connect("../data/processed/features.db")

matrix_df.to_sql('user_item_matrix', conn, if_exists='replace', index=False)
movie_idx_df.to_sql('movie_index', conn, if_exists='replace', index=False)
title_lookup_df.to_sql('title_lookup', conn, if_exists='replace', index=False)

conn.close()
print("\nSaved tables to SQLite database: ../data/processed/features.db")


Saved tables to SQLite database: ../data/processed/features.db


## Summary

**What we saved to SQLite:**

| Table | Description |
|-------|-------------|
| `user_item_matrix` | Movies × users rating matrix |
| `movie_index` | Maps movie_id → matrix row index |
| `title_lookup` | Maps movie titles → movie_id |

**Next step:** Load this in `03_modeling.ipynb` to build the KNN recommender